In [1]:
import os

In [2]:
%pwd

'd:\\StyleConsult\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\StyleConsult'

In [5]:
import dagshub
dagshub.init(repo_owner='RagalahariAkula-42', repo_name='StyleConsult', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "RagalahariAkula-42/StyleConsult"

Repository RagalahariAkula-42/StyleConsult initialized!

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    validation_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    params_seed: int

In [9]:
from gender_classifier.constants import *
from gender_classifier.utils.common import read_yaml, create_directories,save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            validation_data="artifacts/data_ingestion/Validation",
            mlflow_uri="https://dagshub.com/RagalahariAkula-42/StyleConsult.mlflow",
            all_params=self.params,
            params_image_size=self.params.im_shape,
            params_batch_size=self.params.BATCH_SIZE,
            params_seed=self.params.seed
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            rotation_range=30,
            shear_range=0.3,
            zoom_range=0.3
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size,
            batch_size=self.config.params_batch_size,
            seed=self.config.params_seed
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            class_mode='binary',
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="gender_classifier")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-06-04 02:20:27,908: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-04 02:20:27,911: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-04 02:20:27,912: INFO: common: created directory at: artifacts]
Found 11649 images belonging to 2 classes.
243/243 [==============================] - 169s 692ms/step - loss: 0.5325 - acc: 0.7753
[2024-06-04 02:23:17,630: INFO: common: json file saved at: scores.json]


2024/06/04 02:23:18 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-06-04 02:23:19,735: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 6). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Lahari\AppData\Local\Temp\tmpwdrqrxyo\model\data\model\assets
[2024-06-04 02:23:20,225: INFO: builder_impl: Assets written to: C:\Users\Lahari\AppData\Local\Temp\tmpwdrqrxyo\model\data\model\assets]


c:\Users\Lahari\anaconda3\envs\faceshape\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'gender_classifier'.
2024/06/04 02:23:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: gender_classifier, version 1
Created version '1' of model 'gender_classifier'.
